In [1]:
import os
from typing import Any

import cv2
import dbow
import numpy as np
import training
from const import DATABASE_CACHE_PATH, IMAGE_NAMES_CACHE_PATH
from preview import preview_mached_images


class ImageMatcher:
    """
    A class to match images using ORB descriptors and a pre-trained database.

    Attributes:
    -----------
    db : dbow.Database
        The database containing ORB descriptors.
    image_names : np.ndarray
        An array of image names corresponding to the database entries.
    orb : Any
        The ORB detector and descriptor extractor.

    Methods:
    --------
    __init__(image_names_path: str = IMAGE_NAMES_CACHE_PATH, db_path: str = DATABASE_CACHE_PATH):
        Initializes the ImageMatcher with the given image names and database paths.

    find_matched_images(image_path: str, result_num: int = 5):
        Finds and returns the top matched images for the given image path.
    """

    db: dbow.Database
    images_names: np.ndarray
    orb: Any

    def __init__(
        self,
        image_names_path: str = IMAGE_NAMES_CACHE_PATH,
        db_path: str = DATABASE_CACHE_PATH,
    ):

        self.images_names = np.load(image_names_path)
        if not os.path.exists(DATABASE_CACHE_PATH):
            raise ValueError("Database doesn't exist")
        self.db = training.database(db_path)
        self.orb = cv2.ORB_create()

    def find_matched_images(
        self, image_path: str, result_num: int = 5
    ) -> list[tuple[str, float]]:
        image = cv2.imread(image_path)
        _, descs = self.orb.detectAndCompute(image, None)
        descs = [dbow.ORB.from_cv_descriptor(desc) for desc in descs]
        scores: list[float] = self.db.query(descs)
        top_indices = sorted(
            range(len(scores)), key=lambda i: scores[i], reverse=True
        )[:result_num]

        return [(self.images_names[i], scores[i]) for i in top_indices]


if __name__ == "__main__":
    # 1. 設定使用者影像目錄，並加載影像列表
    user_image_dir = os.path.join(os.getcwd(), "data", "user_images")
    user_images = [
        os.path.join(user_image_dir, filename)
        for filename in [
            "validation_002.jpg",
            "validation_012.png",
            "validation_022.jpg",
            "validation_032.jpg",
            "validation_042.jpg",
        ]
    ]

    # 2. 使用 ImageMatcher 進行影像匹配
    image_matcher = ImageMatcher(IMAGE_NAMES_CACHE_PATH, DATABASE_CACHE_PATH)
    matched_images_names = [
        image_matcher.find_matched_images(image, 1)[0][0]
        for image in user_images
    ]

    # 3. 從CSV中提取影像對應的坐標（XYZ）
    img_data = ImageData(GROUND_TRUTH_PATH)
    localised_coordinates = [
        img_data.name2coord(img) for img in matched_images_names
    ]

    # 4. 使用DBSCAN進行聚類，並計算聚類中心點
    clustered_coordinates = localisation(localised_coordinates)

    # 5. 繪製圖像：包括所有匹配影像點和聚類中心點
    wall_coordinates = pd.read_csv(WALL_COORDINATES_PATH)
    points_with_labels = [
        (x, y, label) for (x, y, z), label in zip(localised_coordinates, [point[2] for point in clustered_coordinates])
    ]
    cluster_centers = [(x, y) for x, y in clustered_coordinates]
    
    # 6. 可視化結果
    scatterplot([tuple(x) for x in wall_coordinates[["x", "y"]].values], points_with_labels, cluster_centers)


ModuleNotFoundError: No module named 'dbow'

In [2]:
pip install pydbow

  Using cached pydbow-0.1.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.
